In [1]:
import pandas as pd
from pyspi.calculator import Calculator
import numpy as np
from copy import deepcopy
import glob
import os
import random
import neuromaps
from neuromaps.parcellate import Parcellater
from nibabel import freesurfer as fs
import nibabel as nib
from neuromaps.datasets import fetch_annotation
from pathlib import Path
from scipy.spatial.distance import cdist
from scipy.stats import spearmanr, wilcoxon, mannwhitneyu
import statsmodels.stats.multitest
from surfplot import Plot
from surfplot.datasets import load_example_data
from neuromaps.datasets import fetch_fslr

# Set seed to 127
random.seed(127)

# Add connectome workbench to path
os.environ['PATH'] = os.environ['PATH'] + ':/Applications/workbench/bin_macosx64'

%load_ext rpy2.ipython

In [2]:
%%R 

suppressPackageStartupMessages({
    library(broom)
    library(circlize)
    library(ComplexHeatmap)
    library(cowplot)
    library(dendextend)
    library(FactoMineR)
    library(GGally)
    library(ggseg)
    library(glue)
    library(grid)
    library(patchwork)
    library(see)
    library(tidyverse)
})

# Set cowplot theme
theme_set(theme_cowplot())

In [24]:
data_path = "/Users/abry4213/data/HCP100/"


### UCLA CNP ###
UCLA_CNP_metadata = pd.read_feather(f"{data_path}/time_series_features/UCLA_CNP_sample_metadata_filtered.feather")
UCLA_CNP_Pearson_HoFC = (pd.read_csv(f"{data_path}/time_series_features/UCLA_CNP_Pearson_HoFC.csv")
                         .merge(UCLA_CNP_metadata, on="Sample_ID", how="left"))

UCLA_CNP_Pearson_HoFC_mean = (UCLA_CNP_Pearson_HoFC
                                  .groupby(['Base_Region', 'Diagnosis'])['Pearson_R']
                                  .mean()
                                  .reset_index()
                                  .rename(columns={"Pearson_R": "Mean_Pearson_R"}))

In [29]:
%%R -i UCLA_CNP_Pearson_HoFC_mean

min_Pearson_R = min(UCLA_CNP_Pearson_HoFC_mean$Mean_Pearson_R)
max_Pearson_R = max(UCLA_CNP_Pearson_HoFC_mean$Mean_Pearson_R)

### Pearson FC in brain ###
dx_plot_list <- list() 
for (dx in unique(UCLA_CNP_Pearson_HoFC_mean$Diagnosis)) {
    dx_plot <- UCLA_CNP_Pearson_HoFC_mean %>%
      filter(Diagnosis==dx) %>%
      mutate(label = glue("lh_{Base_Region}")) %>%
      left_join(., as_tibble(dk)) %>%
      ggseg(atlas = dk, mapping = aes(fill = Mean_Pearson_R),
            position = "stacked", colour = "black", hemisphere="left",
            linewidth=0.5) +
      theme_void() +
      ggtitle(dx) +
      labs(fill = "Mean Pearson R") +
      theme(plot.title = element_text(face='bold', size=14, hjust=0.5),
            legend.key.width  = unit(3, "lines"),
            legend.key.height  = unit(0.75, "lines"),
            legend.position = "bottom") +
      scale_fill_viridis_c(na.value = "white", limits=c(min_Pearson_R, max_Pearson_R))
    dx_plot_list[[dx]] <- dx_plot
}

wrap_plots(dx_plot_list, ncol=1) +
       plot_layout(guides = "collect", axis_titles = "collect") & 
       theme(legend.position = 'bottom')

ggsave("../plots/neuropsych_cases/UCLA_CNP_Pearson_HoFC_mean_brains.svg", width=4, height=8, dpi=300, units='in')

Joining with `by = join_by(label)`
Joining with `by = join_by(label)`
Joining with `by = join_by(label)`
Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'


## How do Pearson-based HoFC values differ between Controls vs. SCZ, BP, and ADHD?

In [44]:
UCLA_CNP_Pearson_HoFC_mean.head()

,Base_Region,Diagnosis,Mean_Pearson_R
0,bankssts,ADHD,0.288650
1,bankssts,BP,0.242789
2,bankssts,Control,0.348447
3,bankssts,SCZ,0.278980
4,caudalanteriorcingulate,ADHD,0.360386


In [72]:
%%R -i UCLA_CNP_Pearson_HoFC_mean 


min_Pearson_R = min(UCLA_CNP_Pearson_HoFC_mean$Mean_Pearson_R)
max_Pearson_R = max(UCLA_CNP_Pearson_HoFC_mean$Mean_Pearson_R)

UCLA_CNP_Pearson_HoFC_mean %>% 
    pivot_wider(names_from="Diagnosis", values_from="Mean_Pearson_R") %>%
    pivot_longer(cols=c(-Base_Region, -Control), names_to="Diagnosis", values_to="Mean_Pearson_R") %>%
    mutate(Diagnosis = factor(Diagnosis, levels=c("SCZ", "BP", "ADHD"))) %>%
    ggplot(data=., mapping=aes(x=Mean_Pearson_R, y=Control, fill=Control)) +
    geom_point(color='black', shape=21, size=3) +
    scale_fill_viridis_c() +
    xlab("Mean Pearson HoFC in Diagnostic Group") +
    ylab("Mean Pearson HoFC in Control Group") +
    facet_wrap(Diagnosis ~ ., scales="free", ncol=1) +
    # geom_smooth(method="lm", se=FALSE, color='black', linewidth=0.8) +
    geom_abline(intercept=0, slope=1, color='black', linewidth=0.8) +
    scale_x_continuous(limits=c(0, max_Pearson_R)) +
    scale_y_continuous(limits=c(0, max_Pearson_R)) +
    ggpubr::stat_cor(method="spearman", cor.coef.name="rho", aes(label=..r.label..), size=6, label.x=0.05, label.y=0.7) +
    theme(strip.background = element_blank(),
            strip.text.x = element_text(face='bold', size=18),
            axis.title = element_text(size=20),
            axis.text = element_text(size=18),
          legend.position = "none")
ggsave("../plots/neuropsych_cases/UCLA_CNP_Pearson_HoFC_dx_vs_control_scatters.svg", width=3, height=8, dpi=300, units='in')

In addition: Warning messages:
1: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
2: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
3: The following aesthetics were dropped during statistical transformation: fill.
ℹ This can happen when ggplot fails to infer the correct grouping structure in
  the data.
ℹ Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 


In [35]:
# Subtract mean Pearson R from ADHD, BP, or SCZ vs. Control
dx_HoFC_diffs_from_controls = (UCLA_CNP_Pearson_HoFC_mean
 .pivot(index='Base_Region', columns='Diagnosis', values='Mean_Pearson_R')
 .reset_index()
 .melt(id_vars=['Base_Region', 'Control'], var_name='Diagnosis', value_name='Mean_Pearson_R')
 .assign(DX_minus_Control = lambda x: x['Mean_Pearson_R'] - x['Control'])
 .drop_duplicates(subset=['Base_Region', 'Diagnosis', 'DX_minus_Control'])
 [['Base_Region', 'Diagnosis', 'DX_minus_Control']]
)

dx_HoFC_diffs_from_controls

,Base_Region,Diagnosis,DX_minus_Control
0,bankssts,ADHD,-0.059797
1,caudalanteriorcingulate,ADHD,-0.014918
2,caudalmiddlefrontal,ADHD,0.009903
3,cuneus,ADHD,-0.004640
4,entorhinal,ADHD,-0.040929
...,...,...,...
97,superiorparietal,SCZ,0.007352
98,superiortemporal,SCZ,-0.108798
99,supramarginal,SCZ,-0.072437
100,temporalpole,SCZ,0.112878


In [43]:
%%R -i dx_HoFC_diffs_from_controls

min_diff = min(dx_HoFC_diffs_from_controls$DX_minus_Control)
max_diff = max(dx_HoFC_diffs_from_controls$DX_minus_Control)

### Pearson FC in brain ###
dx_plot_list <- list() 
for (dx in unique(dx_HoFC_diffs_from_controls$Diagnosis)) {
    dx_plot <- dx_HoFC_diffs_from_controls %>%
      filter(Diagnosis==dx) %>%
      mutate(label = glue("lh_{Base_Region}")) %>%
      left_join(., as_tibble(dk)) %>%
      ggseg(atlas = dk, mapping = aes(fill = DX_minus_Control),
            position = "stacked", colour = "black", hemisphere="left",
            linewidth=0.5) +
      theme_void() +
      ggtitle(dx) +
      labs(fill = "Pearson HoFC, DX minus Controls") +
      theme(plot.title = element_text(face='bold', size=14, hjust=0.5),
            legend.key.width  = unit(3, "lines"),
            legend.key.height  = unit(0.7, "lines"),
            legend.position = "bottom") +
      scale_fill_gradient2(low = "#0F1F91", mid = "white", high = "red", midpoint=0, 
                           limits=c(min_diff, max_diff), na.value = "white")
    dx_plot_list[[dx]] <- dx_plot
}

wrap_plots(dx_plot_list, ncol=1) +
       plot_layout(guides = "collect", axis_titles = "collect") & 
       theme(legend.position = 'bottom')

ggsave("../plots/neuropsych_cases/UCLA_CNP_Pearson_HoFC_dx_vs_control_brains.svg", width=4, height=6, dpi=300, units='in')

Joining with `by = join_by(label)`
Joining with `by = join_by(label)`
Joining with `by = join_by(label)`


merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
merging atlas and data by 'label', 'atlas', 'type', 'hemi', 'side', 'region', 'roi'
